# Event Detectors in Orekit

## Authors

Lots of parts are directly from the orekit documentation on [propagation](https://www.orekit.org/site-orekit-10.1/architecture/propagation.html), with some updates, simplifications and Pythonification by Petrus Hyvönen, SSC

## Learning Goals
* *What are Event Detectors*: Why are these useful
* *How do I use Event Detectors*: How is it implemented in Orekit and Python

## Keywords
orekit, propagation, event detectors

Initialize orkit and bring up the python-java interface

In [3]:
import orekit_jpype
vm = orekit_jpype.initVM()

JVM already started, resuming on started JVM


Now set up the pointer to the orekit-data.zip file, using one of the helper files. The file should be in current directory if not specified otherwise.

In [4]:
from orekit_jpype.pyhelpers import setup_orekit_data, absolutedate_to_datetime
setup_orekit_data()

Now we are set up to import and use objects from the orekit library. Packages can be imported as they were native Python packages

# Event Detectors

_Before starting this introduction, please make sure you have refreshed the tutorials on Orbit Definition and Propagation._

The propagators in Orekit is part of an architecture that supports detecting certain discrete conditions that occur during the propagation. This can be that a spacecraft enters eclipse, becomes visible from a ground station, crosses the perigee or a number of other interesting things that may occur during the orbit.

This feature is activated by registering EventDetectors to the propagator. All proppagators in Orekit supports the EventDetector mechanism.

Users can define their own EventDetectors but there are also several predefined EventDetectors
already available, amongst which :

- a simple DateDetector, which is simply triggered at a predefined date, and can be reset to add new dates on the run (which is useful to set up delays starting when a previous event is been detected)

- an ElevationDetector, which is triggered at raising or setting time of a satellite with respect to a ground point, taking atmospheric refraction into account and either constant elevation or ground mask when threshold elevation is azimuth-dependent

- an ElevationExtremumDetector, which is triggered at maximum (or minimum) satellite elevation with respect to a ground point

- an AltitudeDetector which is triggered when satellite crosses a predefined altitude limit and can be used to compute easily operational forecasts

- a FieldOfViewDetector which is triggered when some target enters or exits a satellite sensor Field Of View (any shape),
- a CircularFieldOfViewDetector which is triggered when some target enters or exits a satellite sensor Field Of View (circular shape),
- a FootprintOverlapDetector which is triggered when a sensor Field Of View (any shape, even split in non-connected parts or containing holes) overlaps a geographic zone, which can be non-convex, split in different sub-zones, have holes, contain the pole,
- a GeographicZoneDetector, which is triggered when the spacecraft enters or leave a zone, which can be non-convex, split in different sub-zones, have holes, contain the pole,
- a GroundFieldOfViewDetector, which is triggered when the spacecraft enters or leave a ground based Field Of View, which can be non-convex, split in different sub-zones, have holes,
- an EclipseDetector, which is triggered when some body enters or exits the umbra or the penumbra of another occulting body,
- an ApsideDetector, which is triggered at apogee and perigee,
- a NodeDetector, which is triggered at ascending and descending nodes,
- a PositionAngleDetector, which is triggered when satellite angle on orbit crosses some value (works with either anomaly, latitude argument or longitude argument and with either true, eccentric or mean angles),
- LatitudeCrossingDetector, LatitudeExtremumDetector, LongitudeCrossingDetector, LongitudeExtremumDetector, which are triggered when satellite position with respect to central body reaches some predefined values,
- an AlignmentDetector, which is triggered when satellite and some body projected in the orbital plane have a specified angular separation (the term AlignmentDetector is clearly a misnomer as the angular separation may be non-zero),
- an AngularSeparationDetector, which is triggered when angular separation between satellite and some beacon as seen by an observer goes below a threshold. The beacon is typically the Sun, the observer is typically a ground station
- An EventShifter is also provided in order to slightly shift the events occurrences times. A typical use case is for handling operational delays before or after some physical event really occurs.

An EventSlopeFilter is provided when user is only interested in one kind of events that occurs in pairs like raising in the raising/setting pair for elevation detector, or eclipse entry in the entry/exit pair for eclipse detector. The filter does not simply ignore events after they have been detected, it filters them before they are located and hence save some computation time by not doing an accurate search for events that will ultimately be ignored.

An EventEnablingPredicateFilter is provided when user wants to filter out some events based on an external condition set up by a user-provided enabling predicate function. This allow for example to dynamically turn some events on and off during propagation or to set up some elaborate logic like triggering on elevation first time derivative (i.e. one elevation maximum) but only when elevation itself is above some threshold. 

A BooleanDetector is provided to combine several other detectors with boolean operators and, or and not. This allows for example to detect when a satellite is both visible from a ground station and out of eclipse.

In [5]:
from org.orekit.orbits import KeplerianOrbit, PositionAngleType
from org.orekit.propagation.analytical import KeplerianPropagator
from org.orekit.time import AbsoluteDate, TimeScalesFactory
from org.orekit.utils import Constants, IERSConventions
from org.orekit.frames import FramesFactory
from org.orekit.bodies import OneAxisEllipsoid, CelestialBodyFactory

In [6]:
from math import radians, degrees
import pandas as pd

In [7]:
utc = TimeScalesFactory.getUTC()

Let us do a small example, based on the orbit we used in the Propagation tutorial.

In [8]:
ra = 500 * 1000         #  Apogee
rp = 400 * 1000         #  Perigee
i = radians(87.0)      # inclination
omega = radians(20.0)   # perigee argument
raan = radians(10.0)  # right ascension of ascending node
lv = radians(0.0)    # True anomaly

epochDate = AbsoluteDate(2020, 1, 1, 0, 0, 00.000, utc)
initial_date = epochDate

a = (rp + ra + 2 * Constants.WGS84_EARTH_EQUATORIAL_RADIUS) / 2.0    
e = 1.0 - (rp + Constants.WGS84_EARTH_EQUATORIAL_RADIUS) / a

## Inertial frame where the satellite is defined
inertialFrame = FramesFactory.getEME2000()

## Orbit construction as Keplerian
initialOrbit = KeplerianOrbit(a, e, i, omega, raan, lv,
                              PositionAngleType.TRUE,
                              inertialFrame, epochDate, Constants.WGS84_EARTH_MU)
initialOrbit

<KeplerianOrbit: Keplerian parameters: {a: 6828137.0; e: 0.007322641593160761; i: 86.99999999999999; pa: 20.0; raan: 10.0; v: 0.0;}>

In [9]:
propagator = KeplerianPropagator(initialOrbit)

## Adding Event Detectors

In the first example we create an EventDetector for eclipse, when the satellite is not illuminated by the Sun, and is in the full shadow of the Earth.

In [10]:
ITRF = FramesFactory.getITRF(IERSConventions.IERS_2010, True)
earth = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS, 
                         Constants.WGS84_EARTH_FLATTENING, 
                         ITRF)

In [11]:
sun = CelestialBodyFactory.getSun()
sunRadius = 696000000.0

In [12]:
from org.orekit.propagation.events import EclipseDetector, EventsLogger
from org.orekit.propagation.events.handlers import ContinueOnEvent

The EclipseDetector class is documented at the [Orekit API](https://www.orekit.org/site-orekit-latest/apidocs/org/orekit/propagation/events/EclipseDetector.html) and will detect entering and leaving the full shadow (Umbra) or when some part of the Sun is covered by Earth (Penumbra). 

In the detector, we can also set which EventHandler that we want to use, we can also write our own. In this case we use an EventHandler that will just let the propagator continue after an event has been detected.

In [13]:
eclipse_detector = EclipseDetector(sun, sunRadius, earth).withUmbra().withHandler(ContinueOnEvent())

There are several ways to collect these events when they happen, one of the ways is to use an Orekit EventsLogger that will store the events during the propagation.

In [14]:
logger = EventsLogger()
logged_detector = logger.monitorDetector(eclipse_detector)

We add the eclipse detector, together with the eventslogger to our propagator.

In [15]:
propagator.addEventDetector(logged_detector)

The propagation is executed in same way as in previous examples.

In [16]:
state = propagator.propagate(initial_date, initial_date.shiftedBy(3600.0 * 24))
state.getDate()

<AbsoluteDate: 2020-01-02T00:00:00.000Z>

Now we can fetch the events that the logger found.

In [17]:
events = logger.getLoggedEvents()
events.size()

32

This is a code snippet that goes through the events and store them in a Pandas DataFrame that is very useful for handling tables in Python. In this, the dates are converted also to Python DateTime objects. Please note that if any further use of the data in Orekit is to be done, it is advisable also to save the Orekit AbsoluteDate.

In [18]:
start_time = None
result = []

for event in logger.getLoggedEvents():
   
    if not event.isIncreasing():
        start_time = event.getState().getDate()
    elif start_time:
        stop_time = event.getState().getDate()
        result.append({    "Start":absolutedate_to_datetime(start_time), 
                    "Stop":absolutedate_to_datetime(stop_time),     
                    "EclipseDuration": stop_time.durationFrom(start_time)/60})
        start_time = None
result_df = pd.DataFrame.from_dict(result)
result_df

,Start,Stop,EclipseDuration
0,2020-01-01 00:08:45.102786,2020-01-01 00:26:49.761987,18.077653
1,2020-01-01 01:42:22.923760,2020-01-01 02:00:27.061606,18.068964
2,2020-01-01 03:16:00.738930,2020-01-01 03:34:04.369297,18.060506
3,2020-01-01 04:49:38.548240,2020-01-01 05:07:41.685056,18.052280
4,2020-01-01 06:23:16.351633,2020-01-01 06:41:19.008873,18.044287
5,2020-01-01 07:56:54.149054,2020-01-01 08:14:56.340740,18.036528
6,2020-01-01 09:30:31.940446,2020-01-01 09:48:33.680648,18.029003
7,2020-01-01 11:04:09.725759,2020-01-01 11:22:11.028586,18.021714
8,2020-01-01 12:37:47.504940,2020-01-01 12:55:48.384545,18.014660
9,2020-01-01 14:11:25.277940,2020-01-01 14:29:25.748514,18.007843


# Exercise 1

In this case you are to calculate the ground station visibilities for the above orbit, with a minimum elevation of 5 degrees above the horizon. The detector to use is the [ElevationDetector](https://www.orekit.org/site-orekit-latest/apidocs/org/orekit/propagation/events/ElevationDetector.html). Create a similar pandas table as above of the start / stop time of the visibilities.

# Exercise 2

The satellite you are simulating does not have any batteries onboard, so it needs to be fully in sunlight to operate. Perform a calculation of when your ground station has visibility to the spacecraft, and that the spacecraft is in full sunlight. 

Hint: Check the BooleanDetector, but there are other ways to do this too!
